# ZUMI MAIN (cleaned)

In [ ]:
from zumi.zumi import Zumi
import time

import csv
from zumi.util.screen import Screen

from zumi.util.camera import Camera
from zumi.util.vision import Vision

from zumi.personality import Personality
from datetime import datetime

from zumi.protocol import Note

zumi = Zumi()
zumi.reset_gyro()
camera = Camera()
screen = Screen()
personality = Personality(zumi, screen)
vision = Vision()

screen.happy()

#GLOBAL VARs
whiteTreshold = 150
circRounds = 1
speed = 15
jump_back_threshold = 20


face_count = 0


def corr_line(heading):
    head = heading
    ir_readings = zumi.get_all_IR_data()
    left_ir_bottom = ir_readings[3]
    right_ir_bottom = ir_readings[1]

    if left_ir_bottom > whiteTreshold and right_ir_bottom <= whiteTreshold:
        head += 5
    elif right_ir_bottom > whiteTreshold and left_ir_bottom <= whiteTreshold:
        head -= 5
    elif left_ir_bottom <= whiteTreshold and right_ir_bottom <= whiteTreshold:
        zumi.stop()
        head = "weiss"

    return head
    
def smooth(raw_values):
    smoothed = []
    for i in range(len(raw_values)):
        if i < 2:  # Handle first two elements
            window = raw_values[0:i+1]
        else:
            window = raw_values[i-2:i+1]
        avg = sum(window) / len(window)
        smoothed.append(avg)
    return smoothed

def search_curve(zumi):
    zumi.stop()
    print("Linie verloren – starte gezielte Suche...")

    zumi.reset_gyro()
    time.sleep(0.1)

    # === Suche nach Westen/links (90 Grad) ===
    zumi.turn(90)
    time.sleep(0.3)
    ir_readings = zumi.get_all_IR_data()
    right_ir_bottom = ir_readings[1]
    left_ir_bottom = ir_readings[3]

    if left_ir_bottom > whiteTreshold and right_ir_bottom > whiteTreshold:
        print("Linie im Westen gefunden.")
        zumi.reset_gyro()
        return 0
    if left_ir_bottom > whiteTreshold:
        # Versuch Feinjustierung nach links
        print("Linker Sensor sieht Linie – feinjustiere weiter nach links...")
        zumi.turn_left(20)
        time.sleep(0.3)
        ir_readings = zumi.get_all_IR_data()
        if ir_readings[1] > whiteTreshold and ir_readings[3] > whiteTreshold:
            print("Linie nach Feinjustierung im westen gefunden.")
            zumi.reset_gyro()
            return 0

    # === Suche nach Osten/rechts (-90 Grad) ===
    zumi.turn(-90)  # -90 zurück + 20 zusätzliche Korrekturrichtung
    time.sleep(0.3)
    ir_readings = zumi.get_all_IR_data()
    right_ir_bottom = ir_readings[1]
    left_ir_bottom = ir_readings[3]

    if left_ir_bottom > whiteTreshold and right_ir_bottom > whiteTreshold:
        print("Linie im Osten gefunden.")
        zumi.reset_gyro()
        return 0
    if right_ir_bottom > whiteTreshold:
        # Feinjustierung weiter nach rechts
        print("Rechter Sensor sieht Linie – feinjustiere weiter nach rechts...")
        zumi.turn_right(20)
        time.sleep(0.3)
        ir_readings = zumi.get_all_IR_data()
        if ir_readings[1] > whiteTreshold and ir_readings[3] > whiteTreshold:
            print("Linie nach Feinjustierung im Osten gefunden.")
            zumi.reset_gyro()
            return 0

    # === Keine Linie gefunden ===
    print("Keine Linie erkannt.")
    return None

def follow_circle(zumi, direction="right"):
    heading = 0
    turn_count = 0
    max_turns = 4 * circRounds
    rev = "right"

    while True:
        # === 3. Linienfolge wie gehabt ===
        heading = corr_line(heading)
        if heading == "weiss":
            print("Linie verloren – Turn QR Circle richtung...")

            if direction == "right":
                zumi.turn(-90)  # -90 zurück
                time.sleep(0.3)
                zumi.reset_gyro()
                new_heading = 0
                rev = "left"

            elif direction == "left":
                zumi.turn(90)  # -90 zurück
                time.sleep(0.3)
                zumi.reset_gyro()
                new_heading = 0
                rev = "right"

            turn_count += 1
            print("Abbiegevorgang:", turn_count, "von", max_turns)

            if new_heading is not None:
                heading = new_heading
                if turn_count >= max_turns:
                    while True:
                        print("LOOP AFTER CIRCLE")
                        heading = corr_line(heading)
                        if heading == "weiss":
                            print("Weiße Fläche erkannt")
                            if rev == "left":
                                zumi.turn(90)
                                zumi.reset_gyro()
                            elif rev == "right":
                                zumi.turn(-90)
                                zumi.reset_gyro()
                            return 0
                        
                        zumi.go_straight(10, heading)  # langsam geradeaus
                        time.sleep(0.005)
                continue
            else:
                print("Suche gescheitert – Programmabbruch.")
                break
        
        print("HEADING CIRCLE FOLLOW END: ", heading)
        zumi.go_straight(speed, heading)
        time.sleep(0.02)

    zumi.stop()
    

def follow_line(zumi, abs_ir_threshold=50):
    heading = 0
    start_time = time.time()
    print("HEADING START: ", heading)

    # Historie mit Startwerten füllen
    ir_readings = zumi.get_all_IR_data()
    right_ir_history = [ir_readings[0]] * 10
    left_ir_history = [ir_readings[5]] * 10

    while True:
        ir_readings = zumi.get_all_IR_data()
        left_ir_front = ir_readings[5]
        right_ir_front = ir_readings[0]

        # === 1. Sofortiger IR-Stopp bei direkter Objekterkennung ===
        #if left_ir_front < abs_ir_threshold or right_ir_front < abs_ir_threshold:
        #    zumi.stop()
        #    print("SOFORT-STOPP: Objekt sehr nah (IR-Werte: L=", (left_ir_front), "R=",(right_ir_front))
        #    object_ahead(zumi, screen, personality)
        #    continue

        # Historie aktualisieren
        right_ir_history.append(right_ir_front)
        left_ir_history.append(left_ir_front)
        if len(right_ir_history) > 10:
            print(right_ir_history)
            right_ir_history.pop(0)
        if len(left_ir_history) > 10:
            print(left_ir_history)
            left_ir_history.pop(0)

        # NEW: Apply noise filtering
        right_smoothed = smooth(right_ir_history)
        left_smoothed = smooth(left_ir_history)

        # Kumulierte Deltas berechnen (using smoothed values)
        delta_right_total = sum([
            abs(right_smoothed[i+1] - right_smoothed[i])
            for i in range(len(right_smoothed)-1)
        ])
        delta_left_total = sum([
            abs(left_smoothed[i+1] - left_smoothed[i])
            for i in range(len(left_smoothed)-1)
        ])

        # === 3. Linienfolge wie gehabt ===
        heading = corr_line(heading)
        if heading == "weiss":
            print("Linie verloren – suche neue Richtung...")
            new_heading = search_curve(zumi)
            if new_heading is not None:
                heading = new_heading
                continue
            else:
                print("Suche gescheitert – Programmabbruch.")
                break
                
        # === 2. Reagiere auf kumulativen Sprung ===
        if delta_left_total > jump_back_threshold or delta_right_total > jump_back_threshold:
            zumi.stop()
            print("KUMULATIVER IR-Sprung erkannt! ΔL=" + str(delta_left_total) + " ΔR=" + str(delta_right_total))
            object_ahead(zumi, screen, personality)
            # Historie mit Startwerten füllen
            print("HEADING KUM: ", heading)
            
            ### GEAENDERT
            ir_readings = zumi.get_all_IR_data()
            right_ir_history.clear()
            left_ir_history.clear()
            right_ir_history = [ir_readings[0]] * 10
            left_ir_history = [ir_readings[5]] * 10
            zumi.reset_gyro()
            heading = 0
            print("HEADING NEW KUM: ", heading)
            continue
        
        print("HEADING FOLLOW END: ", heading)
        zumi.go_straight(speed, heading)
        time.sleep(0.02)

    zumi.stop()

def read_qr_message(image):
    qr_code = vision.find_QR_code(image)
    if qr_code is not None:
        message = vision.get_QR_message(qr_code)
        print("QR-Code erkannt:", message)
        return message
    else:
        return None
    
def handle_qr_action(message, zumi, screen, personality):
    emotions = ["Zumi is happy today!", "Zumi is angry today!", "Zumi is celebrating today!"]
    directions = ["Turn Right", "Turn Left", "Right Circle", "Left Circle"]

    if message in directions:
        print("Fahre langsam vor zur Kreuzung...")
        zumi.reset_gyro()
        while True:
            head = corr_line(0)
            if head == "weiss":
                print("Weiße Fläche erkannt – führe QR-Aktion aus")
                break

            zumi.go_straight(10, head)  # langsam geradeaus
            time.sleep(0.005)

        if message == "Turn Right":
            print("QR-Aktion: Rechts abbiegen")
            zumi.turn_right(-90)

        elif message == "Turn Left":
            print("QR-Aktion: Links abbiegen")
            zumi.turn_left(90)


        elif message == "Right Circle":
            print("QR-Aktion: Rechtskreis")
            follow_circle(zumi, direction="right")

        elif message == "Left Circle":
            print("QR-Aktion: Linkskreis")
            follow_circle(zumi, direction="left")


    elif message in emotions:
        print("Fahre langsam zur Zielzone...")
        zumi.reset_gyro()
        while True:
            head = corr_line(0)
            if head == "weiss":
                print("Weiße Fläche erkannt – führe QR-Aktion aus")
                break

            zumi.go_straight(10, head)  # langsam geradeaus
            time.sleep(0.005)

        while True:
            head = corr_line(0)
            if head == "weiss":
                print("Weiße Zielfläche erreicht – führe Emotion aus")
                break

            zumi.go_straight(10, head)
            time.sleep(0.05)

        if message == "Zumi is happy today!":
            print("QR-Aktion: Freude anzeigen")
            screen.happy()
            personality.happy()

        elif message == "Zumi is angry today!":
            print("QR-Aktion: Wut anzeigen")
            screen.angry()
            personality.angry()

        elif message == "Zumi is celebrating today!":
            print("QR-Aktion: Feier-Modus")
            personality.celebrate()

        print("Zumi ist am Ziel angekommen und bleibt stehen.")
        while True:
            zumi.stop()
            time.sleep(1)  # Endlosschleife = vollständiger Stopp


    elif message == "Stop":
        print("QR-Aktion: Stop – Zumi hält an")
        zumi.stop()
        time.sleep(1.5) # hier einstellen wie lange gestoppt wird
        return

    # wurde entfernt aus den missions
    #elif message == "Park":
    #    None
    # wurde entfernt aus den missions
    #elif message == "Drive Round Road":
    #    None

    else:
        print("QR-Inhalt unbekannt:", message)
        
    time.sleep(0.5)  # kleine Pause nach der Aktion

def object_ahead(zumi, screen, personality):
    zumi.stop()
    time.sleep(0.7)

    ir_readings = zumi.get_all_IR_data()
    base_left_ir = ir_readings[5]
    base_right_ir = ir_readings[0]

    print("Hindernis erkannt – Stop & Foto")
    zumi.play_note(Note.C4, 500)
    zumi.play_note(Note.C5, 500)  # Spielt mittleres C für 500ms
    zumi.brake_lights_on()
    screen.angry()
    personality.angry()
    time.sleep(0.7)
    zumi.brake_lights_off()
    print("IR vorne: IR0 =", base_right_ir, "IR5 =", base_left_ir)

    try:
        camera.start_camera()
        time.sleep(0.9)
        image = camera.capture()
        camera.show_image(image)
        message = read_qr_message(image)
        time.sleep(1.0)

        if message is not None:
            print("QR erkannt – führe Aktion aus")
            handle_qr_action(message, zumi, screen, personality)
            time.sleep(1.0)
        else:
            print("Kein QR-Code – warte auf Objekt-Entfernung...")

            # Historie für Differenzsprung rückwärts (FIFO-Puffer)
            left_ir_history = [base_left_ir] * 10
            right_ir_history = [base_right_ir] * 10

            time.sleep(0.5)

            while True:
                ir_readings = zumi.get_all_IR_data()
                left_ir = ir_readings[5]
                right_ir = ir_readings[0]

                left_ir_history.append(left_ir)
                right_ir_history.append(right_ir)
                if len(left_ir_history) > 10:
                    left_ir_history.pop(0)
                if len(right_ir_history) > 10:
                    right_ir_history.pop(0)

                # Kumulierte Differenzen berechnen (über Zwischenwerte)
                delta_left_total = sum([
                    left_ir_history[i+1] - left_ir_history[i]
                    for i in range(len(left_ir_history)-1)
                ])
                delta_right_total = sum([
                    right_ir_history[i+1] - right_ir_history[i]
                    for i in range(len(right_ir_history)-1)
                ])

                print("IR: L=" + str(left_ir) + ", R=" + str(right_ir) +
                      " | ΔL=" + str(delta_left_total) + ", ΔR=" + str(delta_right_total))

                if abs(delta_left_total) > jump_back_threshold or abs(delta_right_total) > jump_back_threshold:
                    print("Kumulativer IR-Sprung (Rückgang) erkannt – prüfe auf Gesicht.")
                    #break
                    time.sleep(0.5)
    
                    # Gesichtserkennung nach Objektentfernung
                    print("Prüfe auf Gesicht hinter dem Objekt...")
                    image = camera.capture()
                    camera.show_image(image)
                    face_coords = vision.find_face(image)

                    if face_coords is not None:
                        print("Gesicht erkannt!")
                        #global face_count
                        #face_count += 1
                        screen.draw_text_center("Gesicht erkannt!")
                    else:
                        print("Kein Gesicht erkannt.")

                    break

                time.sleep(0.5)

    finally:
        screen.happy()
        camera.close()
        print("Kamera geschlossen.")
        time.sleep(0.5)

    return True

def drive_course(zumi, total_time=60):

    print("Starte Parkour-Fahrt")
    start_time = time.time()

    while time.time() - start_time < total_time:
        print("Starte neuen Block...")

        # Linien folgen für festgelegte Zeit
        follow_line(zumi)

    zumi.stop()
    print("Parkour abgeschlossen.")

### Aufruf main func

In [ ]:
# Zeitbegrenzung noch auf alt 120 sec = 2 mins, -> neu = 6 mins -> 360
drive_course(zumi, total_time=120)

### Notbremse nach Interrupt

In [ ]:
zumi.stop()

# Ablage Csv loggin func  

Da vermutlich so nicht funktionierendem im code (änderungen object ahead etc) - hab ich aber nicht getestet (SG, 13.05.2025)

## CSV Datei erstellen und schreiben